# Notes on Collaborative Filtering for Implicit Feedback Datasets

# Script

#### Across all x (users) and y (items), find the values of u and i that minimize the summation below.

$\underset{x,y}min\underset{u,i}\sum 
c_{ui} (p_{ui} - x_u^Ty_i)^2 + \lambda
(\underset u \sum \parallel x_u \parallel ^2
+\underset u \sum \parallel y_i \parallel ^2)$

$r_{ui}$ = number of user item interactions

$p_{ui} = 1$ if interaction, 
$p_{ui} = 0$ if no interaction.

$c_{ui} = 1 + \alpha * r_{ui}$

$x_u$ is user vector,
$y_i$ is item vector.


We are taking the squared error of our prediction and 
multiplying by the confidence.

We then have a term to regularize our final x and y vectors, penalizing larger values.

model formulation:

alpha is how we influence our confidence levels. Obviously we get more confident as # of prod_samples increases, but suppose we want to weight higher # of samples much higher, alpha is for that.

They add 1 to each as well, so c


We can't use stochastic gradient descent because of the size of the dataset. (m * n terms)

Therefore we modify the cost function to the Alternating Least Squares.

Hold user vectors or item vectors constant and calculate the global minimum while only changing the other vector.

1. Recompute user factors

    1. item-factors are in $Y$ matrix of $n * f$
    2. Compute $Y^TY$ with $f*f$
    3. For user $u$, define $n*n$ diagonal $C^u$ where $C^u_{ii} = c_{ui}$. This is our confidence matrix for n items, with values only on diagonal
    4. For each user, define $p(u)$ that is all the preferences for said user.
    5. $x_u = (Y^T C^u Y + \lambda I)^{-1}  Y^T C^u p(u)$

2. Recompute user factors

    1. user_factors is $m * f$ matrix $X$
    2. Compute $f * f$ matrix $X ^T X$
    3. For each item $i$, define diagonal $m * m$ matrix $C^i$, where $C_{uu}^i = c_{ui}$
    4. Define $p(i)$ which is the preferences for item i
    5. $y_i = (X^TC^iX + \lambda I)^-1 X^TC^ip(i)$

# Explaining the recommendation

1. Substitute in for user factors the equation for user factors
$x_u = (Y^T C^u Y + \lambda I)^{-1}  Y^T C^u p(u)$

Predicted preference of user $u$ at item $i$ 

$\hat{p}_{ui} = y_i^Tx_u$

Thus

$\hat{p}_{ui} =  y_i^T(Y^T C^u Y + \lambda I)^{-1}  Y^T C^u p(u)$

Denote $f*f$ matrix $(Y^T C^u Y + \lambda I)^{-1}$ as $W^u$

$W^u$ is considered the weight for user $u$
